This notebook is used to fine-tune the visual and textual ensemble architecture. In this mode is saved the best model and in the main notebook of the project is possible to load it.  
To use it, run it after the setup from the main notebook.  
<a href="https://colab.research.google.com/drive/1BOEUiS7uyPme9dhqW388NY3jxHA9JWIg?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

## Initialization
For the setup of the notebook and of the folders refer to the notebook `task1_3`.

In [ ]:
# Mount Google Drive
from google.colab import drive  # import drive from google colab

ROOT = "/content/drive"  # default location for the drive

drive.mount(ROOT)  # we mount the google drive at /content/drive

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/DL/

/content/drive/MyDrive/DL


In [ ]:
%%capture
!pip install "pytorch-lightning" "torchmetrics" "transformers" "prettytable"

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

from PIL import Image

from prettytable import PrettyTable

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.models import resnet50

from torchmetrics import functional

import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModel,
    AutoConfig,
    get_constant_schedule_with_warmup,
)

In [ ]:
seed_everything(21)

Global seed set to 21


21

## Datasets definitions

In [ ]:
DATA_DIR = "/content/drive/MyDrive/DL/SEMEVAL-2021-task6-corpus/data/"

In [ ]:
my_binarizer_task3 = MultiLabelBinarizer()
classes_file = "./SEMEVAL-2021-task6-corpus/techniques_list_task3.txt"
labels_name3 = []
with open(classes_file, "r") as f:
    for line in f.readlines():
        labels_name3.append(line.rstrip())
labels_name3.pop(-1)
labels_name3.sort()  # MultiLabelBinarizer sorts the labels
my_binarizer_task3.fit([labels_name3]);

In [ ]:
# function to load the data
def load_data(data_type, task):
    all_idxs = []
    file_name = "%s_set_task%d.txt" % (data_type, task)
    if task == 3 and data_type == "dev":
        df = pd.read_json(DATA_DIR + "dev_set_task3_labeled.txt")
    else:
        df = pd.read_json(os.path.join(DATA_DIR, file_name))
    all_idxs = df["id"].to_numpy()
    all_data = df["text"].to_numpy()
    if task == 3:
        all_images = df["image"].to_numpy()
        all_labels = my_binarizer_task3.transform(df["labels"])
    else:
        all_images = None
        all_labels = my_binarizer_task1.transform(df["labels"])
    return all_idxs, all_data, torch.tensor(all_labels), all_images

In [ ]:
# Dataset class for single models
class PersTecData(torch.utils.data.Dataset):
    def __init__(self, data_type="training", task=1, tokenizer=None, transforms=None):
        idxs, X, self.y, self.image = load_data(data_type, task)
        self.tokenized = False
        if tokenizer != None:
            self.tokenized = True
            tokenized = tokenizer(
                X.tolist(), padding="max_length", truncation=True, max_length=128
            )
            self.input_ids = torch.tensor(tokenized["input_ids"])
            self.attention_mask = torch.tensor(tokenized["attention_mask"])
        else:
            self.X = X
        self.transforms = transforms
        self.task = task
        self.data_type = data_type

    def __getitem__(self, index):
        if self.task == 3:
            if self.data_type == "training":
                path = DATA_DIR + "training_set_task3/"
            if self.data_type == "dev":
                path = DATA_DIR + "dev_set_task3_labeled/"
            if self.data_type == "test":
                path = DATA_DIR + "test_set_task3/"
            image = Image.open(path + str(self.image[index])).convert("RGB")
            image = self.transforms(image)
            if self.tokenized:
                sample = self.input_ids[index]
                mask = self.attention_mask[index]
                label = torch.squeeze(self.y[index])
                return sample, mask, label, image
            else:
                sample = self.X[index]
                label = np.squeeze(self.y[index])
                return sample, label, image
        else:
            if self.tokenized:
                sample = self.input_ids[index]
                mask = self.attention_mask[index]
                label = torch.squeeze(self.y[index])
                return sample, mask, label
            else:
                sample = self.X[index]
                label = np.squeeze(self.y[index])
                return sample, label

    def __len__(self):
        if self.tokenized:
            return self.input_ids.shape[0]
        else:
            return self.X.shape[0]

## Architectures of the models

### `PLMClassifier` class

In [ ]:
class PLMClassifier(pl.LightningModule):
    def __init__(self, plm, output_dim=20):
        super().__init__()
        self.plm = plm
        self.learning_rate = 2e-5
        self.n_warmup_steps = 500
        self.criterion = nn.BCELoss()

    def forward(self, samples, masks):
        x = self.plm(samples, masks)
        return torch.sigmoid(x.logits)

    def training_step(self, batch, batch_idx):
        if TASK == 1:
            batch_ids, batch_mask, labels = batch
        if TASK == 3:
            batch_ids, batch_mask, labels, _ = batch
        preds = self(samples=batch_ids, masks=batch_mask)
        loss = self.criterion(preds, labels.float())
        self.log("train_loss", loss.item())
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        if TASK == 1:
            batch_ids, batch_mask, labels = batch
        if TASK == 3:
            batch_ids, batch_mask, labels, _ = batch
        preds = self(samples=batch_ids, masks=batch_mask)
        val_loss = self.criterion(preds, labels.float())
        self.log("val_loss", val_loss.item())
        return {"loss": val_loss}

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = get_constant_schedule_with_warmup(
            optimizer, num_warmup_steps=self.n_warmup_steps
        )
        return dict(
            optimizer=optimizer, lr_scheduler=dict(scheduler=scheduler, interval="step")
        )

### `EnsembleClassifier` class

In [ ]:
class EnsembleClassifier(pl.LightningModule):
    def __init__(self, models):
        super().__init__()
        self.models = []
        for model in models:
            self.models.append(model)
        self.n_models = len(self.models)

    def forward(self, batch):
        preds = []
        for i, model in enumerate(self.models):
            samples, masks, _ = batch[i]
            device = model.device
            samples = samples.to(device)
            masks = masks.to(device)
            x = model(samples, masks)
            preds.append(x)
        preds = torch.stack(preds)
        pred = torch.mean(preds, axis=0)
        return pred

In [ ]:
class ImageClassifier(pl.LightningModule):
    def __init__(self, num_classes=22, lr=2e-5):
        super().__init__()
        self.lr = lr
        self.model = resnet50(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_classes)
        self.criterion = nn.BCELoss()

    def forward(self, image):
        x = self.model(image)
        return x

    def training_step(self, batch, batch_idx):
        _, _, label, image = batch
        preds = self(image)
        # print(preds.shape)
        # print(label.shape)
        loss = self.criterion(torch.sigmoid(preds), label.float())
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        _, _, label, image = batch
        preds = self.model(image)
        # print(preds.shape)
        # print(label.shape)
        loss = self.criterion(torch.sigmoid(preds), label.float())
        self.log("validation_loss", loss)
        return loss

    def configure_optimizers(self):
        # return optimizer
        optimizer = optim.Adam(self.model.fc.parameters(), lr=self.lr)
        return optimizer

In [ ]:
class LanguageAndVisionEnsemble(pl.LightningModule):
    def __init__(
        self,
        language_classifier,
        vision_classifier,
        num_classes=22,
    ):
        super(LanguageAndVisionEnsemble, self).__init__()
        self.language_classifier = language_classifier
        self.vision_classifier = vision_classifier

    def forward(self, text, mask, image):
        text_preds = self.language_classifier(text, mask)
        image_preds = self.vision_classifier(image)
        preds = torch.mean(torch.stack([text_preds, image_preds]), dim=0)
        return preds

# TASK 3

In [ ]:
TASK = 3
transforms = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
NUM_LABELS = 22

## Datasets creation

### Find mean and std for normalizing the images

In [ ]:
%%capture

transforms = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
dataset_train_temp = PersTecData(
    data_type="training", task=TASK, tokenizer=tokenizer, transforms=transforms
)
train_loader_temp = DataLoader(
    dataset_train_temp, batch_size=8, num_workers=2, pin_memory=True
)

# finding average and std of images
rgb_mean = []
rgb_var = []
for batch in train_loader_temp:
    _, _, _, images = batch
    rgb_mean.append(torch.mean(images, (0, 2, 3)))
    rgb_var.append(torch.square(torch.std(images, (0, 2, 3), unbiased=True)))
rgb_mean = torch.mean(torch.stack(rgb_mean), dim=0)
rgb_std = torch.sqrt(torch.mean(torch.stack(rgb_var), dim=0))

del tokenizer
del dataset_train_temp
del train_loader_temp

In [ ]:
print(rgb_mean)
print(rgb_std)

tensor([0.4695, 0.4162, 0.4042])
tensor([0.3170, 0.3070, 0.3074])


In [ ]:
transforms = T.Compose(
    [
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=rgb_mean, std=rgb_std),
    ]
)

### DeBERTa datasets

In [ ]:
%%capture
tokenizer4 = AutoTokenizer.from_pretrained("microsoft/deberta-base", use_fast=True)
dataset_train3_DeBERTa = PersTecData(
    data_type="training", task=TASK, tokenizer=tokenizer4, transforms=transforms
)
train_loader_DeBERTa3 = DataLoader(
    dataset_train3_DeBERTa, batch_size=8, num_workers=2, pin_memory=True
)

dataset_val3_DeBERTa = PersTecData(
    data_type="dev", task=TASK, tokenizer=tokenizer4, transforms=transforms
)
val_loader_DeBERTa3 = DataLoader(
    dataset_val3_DeBERTa, batch_size=8, num_workers=2, pin_memory=True
)

dataset_test3_DeBERTa = PersTecData(
    data_type="test", task=TASK, tokenizer=tokenizer4, transforms=transforms
)
test_loader_DeBERTa3 = DataLoader(
    dataset_test3_DeBERTa, batch_size=8, num_workers=2, pin_memory=True
)

## Baseline 2 - Visual and Textual models ensemble

## Visual Classifier
Image classifier for images using `ResNet50`.

In [ ]:
classifier = ImageClassifier()
logger = TensorBoardLogger("tb_logs", name="Image_classifier", log_graph=True)

trainer = pl.Trainer(progress_bar_refresh_rate=20, gpus=1, max_epochs=10, logger=logger)
trainer.fit(classifier, train_loader_DeBERTa3, val_loader_DeBERTa3)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/tensorboard.py:251: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  UserWarning,

  | Name      | Type    | Params
------------------

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 21


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Textual classifier

In [ ]:
deberta = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-base", num_labels=NUM_LABELS
)
model_deberta_task3 = PLMClassifier(deberta)

Downloading:   0%|          | 0.00/533M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'pooler.d

In [ ]:
EPOCHS = 10
# define the logger object
logger = TensorBoardLogger("tb_logs", name="DeBERTaClassifier_task3", log_graph=True)
checkpoint_callback6 = ModelCheckpoint(monitor="val_loss")

if torch.cuda.is_available():
    trainer = pl.Trainer(
        devices=1,
        accelerator="auto",
        max_epochs=EPOCHS,
        logger=logger,
        check_val_every_n_epoch=1,
        gradient_clip_val=1.0,
        callbacks=[checkpoint_callback6],
    )
else:
    print("ERROR: a GPU is needed for training")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
# training DeBERTa
trainer.fit(model_deberta_task3, train_loader_DeBERTa3, val_loader_DeBERTa3)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/tensorboard.py:251: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  UserWarning,

  | Name      | Type                             | Params
---------------------------------------------------------------
0 | plm       | DebertaForSequenceClassification | 139 M 
1 | criterion | BCELoss                          | 0     
---------------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
556.837   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 21


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Visual and Textual classifier ensemble

In [ ]:
lv_ensemble = LanguageAndVisionEnsemble(model_deberta_task3, classifier)

## Testing model

### Testing function definition

In [ ]:
def test_classifier_lv(model, data_loader, thresholds, n_models=1):
    model.cuda()
    model.eval()
    micro_f1 = []
    macro_f1 = []
    outputs = []
    true_labels = []
    for i, batch in enumerate(data_loader):
        batch_ids, batch_mask, labels, batch_img = batch
        true_labels.append(labels)
        outputs.append(
            model(batch_ids.cuda(), batch_mask.cuda(), batch_img.cuda())
            .detach()
            .cpu()
        )
    true_labels = torch.cat(true_labels)
    outputs = torch.cat(outputs)
    # print(outputs)
    for threshold in thresholds:
        predictions = torch.greater(outputs, torch.ones(outputs.shape) * threshold)
        # num_pred = [[1 if x else 0 for x in p] for p in predictions]
        # print(np.sum(num_pred,axis=0))
        macro_f1.append(
            f1_score(true_labels, predictions, average="macro", zero_division=1)
        )
        micro_f1.append(
            f1_score(true_labels, predictions, average="micro", zero_division=1)
        )

    return macro_f1, micro_f1

###Validation dataset performance

In [ ]:
thresholds = [x / 10 for x in range(0, 11)]

val_table = PrettyTable(["Model", "Micro F-1", "Macro F-1"])
val_table.title = "Test set results"

macro_f1_lv_val, micro_f1_lv_val = test_classifier_lv(
    lv_ensemble, val_loader_DeBERTa3, thresholds
)

val_table.add_row(
    [
        "LV",
        str(format(max(micro_f1_lv_val), ".5f")),
        str(format(max(macro_f1_lv_val), ".5f")),
    ]
)
print(val_table)

####F1-Micro

In [ ]:
fig = plt.figure(figsize=(15, 5))
thresholds = [x / 10 for x in range(0, 11)]
plt.plot(thresholds, micro_f1_lv_val, label="LV")
plt.legend(loc="best")
plt.title("f1-micro for models")
plt.xlabel("threshold")
plt.ylabel("f1-micro")

####F1-Macro

In [ ]:
fig = plt.figure(figsize=(15, 5))
thresholds = [x / 10 for x in range(0, 11)]
plt.plot(thresholds, macro_f1_lv_val, label="LV")
plt.legend(loc="best")
plt.title("f1-macro for models")
plt.xlabel("threshold")
plt.ylabel("f1-macro")

###Test dataset performance

In [ ]:
thresholds = [x / 10 for x in range(0, 11)]

test_table = PrettyTable(["Model", "Micro F-1", "Macro F-1"])
test_table.title = "Validation set results"

macro_f1_lv, micro_f1_lv = test_classifier_lv(
    lv_ensemble, test_loader_DeBERTa3, thresholds
)

test_table.add_row(
    [
        "LV",
        str(format(max(micro_f1_lv), ".5f")),
        str(format(max(macro_f1_lv), ".5f")),
    ]
)
print(test_table)

####F-1 Micro

In [ ]:
fig = plt.figure(figsize=(15, 5))
thresholds = [x / 10 for x in range(0, 11)]
plt.plot(thresholds, micro_f1_lv, label="LV")
plt.legend(loc="best")
plt.title("f1-micro for models")
plt.xlabel("threshold")
plt.ylabel("f1-micro")

####F-1 Macro

In [ ]:
fig = plt.figure(figsize=(15, 5))
thresholds = [x / 10 for x in range(0, 11)]
plt.plot(thresholds, macro_f1_lv, label="LV")
plt.legend(loc="best")
plt.title("f1-macro for models")
plt.xlabel("threshold")
plt.ylabel("f1-macro")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs